In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

In [ ]:
# Format(s) of the output file(s). Possible formats are: txt, vtt, srt, tsv, csv, json. Several formats can be specified by using commas (ex: "json,vtt,srt"). By default ("all"), all available formats  
import json
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from process_recordings import find_extant_output_files, write_results, process_recordings
from transcript_to_lsl import VideoTranscriptToLabStreamingLayer, process_transcript_to_lsl
# from whisper.utils import read_csv

recordings_dir = Path(r"M:\ScreenRecordings\EyeTrackerVR_Recordings").resolve()
# Define the recordings directory
if isinstance(recordings_dir, str):
    recordings_dir = Path(recordings_dir).resolve()
print(f'processing_recordings for recordings_dir: "{recordings_dir.as_posix()}"...')
# Create output directory
output_dir: Path = recordings_dir.joinpath('transcriptions').resolve()
lsl_converted_streams_output_dir: Path = output_dir.joinpath('LSL_Converted')
lsl_converted_streams_output_dir.mkdir(exist_ok=True)

# output_dir = Path("./transcriptions")

# output_extensions = ['.txt', '.vtt', '.srt', '.tsv', '.csv', '.json']
output_extensions = ['.json']
# output_extensions = ['.csv']

found_output_files: List[Path] = []
for ext in output_extensions:
    found_output_files.extend(output_dir.glob(f"*{ext}"))

    
# found_output_files: List[Path] = find_extant_output_files(output_dir=output_dir, base_name=base_name, output_formats=output_formats)
found_output_files

found_valid_output_files = []
read_valid_output_files_dict = {}

for a_file in found_output_files:
    if a_file.exists() and a_file.is_file():
        # file_contents = json.load(a_file)
        with open(a_file, "r", encoding="utf-8") as js:
            file_contents = json.load(js) # (result, js, indent=2, ensure_ascii=False)
        if file_contents:
            is_ready_for_LSL_stream_export: bool = False
            if len(file_contents['segments']) > 0:
                try:
                    file_contents['segments'] = VideoTranscriptToLabStreamingLayer.add_absolute_timestamps(segments=file_contents['segments'], file_basename=a_file.stem)
                    print(f"\nSuccess! '{a_file.as_posix()}'\n\tProcessed {len(file_contents['segments'])} transcript segments")
                    is_ready_for_LSL_stream_export = True
                except Exception as e:
                    print(f"Failed to parse to LabStreamingLayer for file: '{a_file.as_posix()}' Error: {e}")
                    is_ready_for_LSL_stream_export = False
                    pass

                if is_ready_for_LSL_stream_export:            
                    try:
                        # lsl_stream_output_path = lsl_converted_streams_output_dir / f"{a_file.stem}.lsl.json"
                        lsl_stream_output_path = lsl_converted_streams_output_dir / f"{a_file.stem}.lsl.fif"
                        lsl_stream_output, raw_lsl_stream_output = VideoTranscriptToLabStreamingLayer.create_lsl_stream_data(file_contents['segments'], stream_save_filename=lsl_stream_output_path)                
                        print(f"\tSuccess exporting to LSL Stream! '{lsl_stream_output_path.as_posix()}'")
                    except Exception as e:
                        print(f"\tFailed to export final LabStreamingLayer stream to '{lsl_stream_output_path.as_posix()}' for source file: '{a_file.as_posix()}' Error: {e}")
                        raise
            
            # print(f'file_contents: {file_contents}')    
        
        # _a_read_text: str = a_file.read_text()
        # if _a_read_text:
            
        #     found_valid_output_files.append(a_file)
        #     read_valid_output_files_dict[a_file.name] = _a_read_text
        #     ## actually include the file
        #     try:
        #         df, lsl_data = process_transcript_to_lsl(csv_path=a_file, output_dir=output_dir, video_filename=a_file.name)
        #         print(f"\nSuccess! Processed {len(df)} transcript segments")
        #     except Exception as e:
        #         print(f"Failed to parse to LabStreamingLayer for file: '{a_file.as_posix()}' Error: {e}")
        #         pass

read_valid_output_files_dict

processing_recordings for recordings_dir: "M:/ScreenRecordings/EyeTrackerVR_Recordings"...
Failed to parse to LabStreamingLayer for file: 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/2024-09-11 15-27-36.json' Error: Could not parse datetime from filename '2024-09-11 15-27-36': Filename '2024-09-11 15-27-36' doesn't match expected pattern
Failed to parse to LabStreamingLayer for file: 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/2024-09-11 15-27-36.words.json' Error: Could not parse datetime from filename '2024-09-11 15-27-36': Filename '2024-09-11 15-27-36' doesn't match expected pattern
Failed to parse to LabStreamingLayer for file: 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/2024-09-11 16-02-51.json' Error: Could not parse datetime from filename '2024-09-11 16-02-51': Filename '2024-09-11 16-02-51' doesn't match expected pattern
Failed to parse to LabStreamingLayer for file: 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/2024

l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-01T113802.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"


[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-01T113802.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-01T114601.json'
	Processed 37 transcript segments
Creating RawArray with float64 data, n_channels=1, n_times=1082339
    Range : 0 ... 1082338 =      0.000 ...  1082.338 secs
Ready.
Writing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-01T114601.lsl.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-01T114601.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-01T114601.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-03T230155.json'
	Processed 1 transcript segments
Crea

l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-01T114601.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"
l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-03T230155.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"
l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: Thi

Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-03T230218.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-03T230218.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-03T230218.words.json'
	Processed 226 transcript segments
Creating RawArray with float64 data, n_channels=1, n_times=1953039
    Range : 0 ... 1953038 =      0.000 ...  1953.038 secs
Ready.
Writing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-03T230218.words.lsl.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-03T230218.words.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-03T230218.words.lsl.fif'

...

Writing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-08T124140.words.lsl.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-08T124140.words.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-08T124140.words.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-08T172024.words.json'
	Processed 2 transcript segments
Creating RawArray with float64 data, n_channels=1, n_times=156680
    Range : 0 ... 156679 =      0.000 ...   156.679 secs
Ready.
Writing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-08T172024.words.lsl.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL

l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-08T172024.words.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"
l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-08T173612.words.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"
l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: Runtime

Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T040429.words.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-09T040429.words.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-09T100733.words.json'
	Processed 655 transcript segments
Creating RawArray with float64 data, n_channels=1, n_times=1537620
    Range : 0 ... 1537619 =      0.000 ...  1537.619 secs
Ready.
Writing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T100733.words.lsl.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T100733.words.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-09T100733.wo

l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T100733.words.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"
l:\repos\ExternalTesting\whisper-timestamped\transcript_to_lsl.py:214: RuntimeWarning: This filename (M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T103250.words.lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  file_type = "FIF"


Closing M:\ScreenRecordings\EyeTrackerVR_Recordings\transcriptions\LSL_Converted\Debut_2025-07-09T103250.words.lsl.fif
[done]
	Success exporting to LSL Stream! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/LSL_Converted/Debut_2025-07-09T103250.words.lsl.fif'

Success! 'M:/ScreenRecordings/EyeTrackerVR_Recordings/transcriptions/Debut_2025-07-09T105906.words.json'
	Processed 7 transcript segments
Creating RawArray with float64 data, n_channels=1, n_times=133109
    Range : 0 ... 133108 =      0.000 ...   133.108 secs
Ready.


In [15]:
file_contents['segments'] = VideoTranscriptToLabStreamingLayer.add_absolute_timestamps(segments=file_contents['segments'], file_basename=a_file.stem)

	 

In [20]:
a_segment = file_contents['segments'][0]
a_segment
# a_segment['text']
# a_segment['start']
# a_segment['end']

# file_contents['text']

{'id': 0,
 'seek': 177000,
 'start': 1770.0,
 'end': 1800.0,
 'text': ' A1-A1, A2-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A3-A1, A',
 'tokens': [50363,
  317,
  16,
  12,
  32,
  16,
  11,
  317,
  17,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12,
  32,
  16,
  11,
  317,
  18,
  12

In [ ]:
output_files = process_recordings(recordings_dir=recordings_dir, video_extensions=video_extensions)
print(f'All processing complete! output_files: {output_files}\n\ndone.')
